In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import special
import healpy as hp
import os
import imageio
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

In [2]:
def temporalPlot(pt,Nt,gaps):
    tbeg, tend = 1717.6256+(np.linspace(1666.4384902198801, 2704.3655735533684, 2) + 2455197.5 - 2457023.5 - 0.25)*4
    bins = np.linspace(tbeg,tend,Nt+1)
    centres = 0.5*(bins[1:] + bins[:-1])

    plt.plot(centres,pt)
    
    if type(gaps) != bool:
        for start,end in zip(gaps['tbeg'].values,gaps['tend'].values):
            plt.fill_between([start,end],y1=[-2,-2],y2=[2,2],color='lightblue',lw=0.0)
    plt.ylim([0,1.02])
    plt.xlabel('OBMT (revolutions)')
    plt.ylabel('pt')
    plt.xticks(np.arange(1000,5251,250))
    plt.yticks(np.arange(0.0,1.01,0.1))
    # plt.grid(axis='both',which='both',lw=0.3,color='lightgrey',zorder=0)
    # plt.savefig(directory+'plot_pt.png',dpi=300,facecolor='w',bbox_inches='tight')
  

In [3]:
def spatialPlot(pml):
    hp.mollview(pml, nest=False, hold=True, min=0,max=1,cmap='viridis', cbar=True,coord=['C','G'],notext=True)

In [4]:
def readFile(directory,file):
    try:
        params = pd.read_csv(directory+'OptimizerProperties.dat',skipinitialspace=True,index_col=0,sep=" =",header=None,engine='python').transpose()
        Nt = int(params['Nt'])
        Nm = int(params['Nm'])
        Nl = int(params['Nl'])
    except:
        params = pd.read_csv(directory+'Optimiser_Properties.dat',skipinitialspace=True,sep=",")
        Nt = int(params['Nt'])
        Nm = int(params['Nm'])
        Nl = int(params['Nl'])
    alpha = 0.5*np.log(2)
    a = -3.0
    if type(file) == int:
        file = "TempPositions/TempPosition" + str(file) + "_TransformedParameters.dat"
    xt = pd.read_csv(directory+file,header=None)[0][:Nt].values
    pt = special.expit(xt)
    xml = np.reshape(pd.read_csv(directory+file,header=None)[0][Nt:Nt+Nm*Nl].values,(Nl,Nm)).T
    pml = np.exp(-2.0*alpha*np.exp(-xml))
    pml[xml<a] = np.exp(-2.0*alpha*(1.0-xml[xml<a]+a)*np.exp(-a))
    return [pt,pml]

In [5]:
def baseComparisonPlot(truept,truepml,pt,pml,gaps):
    fig = plt.figure(figsize=(8,8))
    
    ##temporal plot
    fig.add_subplot(2,1,1)
    temporalPlot(truept,len(truept),False)
    temporalPlot(pt,len(pt),False)
    
    
    ##spatial plot
    fig.add_subplot(2,2,3)
    spatialPlot(truepml[0])
    fig.add_subplot(2,2,4)
    spatialPlot(pml[0])
    

In [6]:
def baseGif(saveName,truedirectory,tempdirectory,start,stop):
    
    trueFile = "True_TransformedParameters.dat"
    [truePt,truePml] = readFile(truedirectory,trueFile)
    
    names = []
    for i in range(start,stop):
        print(i)
      
        [pt,pml] = readFile(directory,i)
        baseComparisonPlot(truePt,truePml,pt,pml,False)
        
        tempName = saveName +"_temp" + str(i) + ".png"
        names.append(tempName)
        plt.savefig(tempName)
        plt.close()
        %reset_selective -f pt
        
        # %reset plot
        
    saveName = saveName + ".gif"
    with imageio.get_writer(saveName,mode = 'I') as writer:
        for name in names:
            image = imageio.imread(name)
            writer.append_data(image)
            del image
    for name in names:
        os.remove(name)

In [7]:
dataDir = "../../../Data/TestSets/flat/"
directory = "../../../CodeOutput/TestSets/flat/"
baseGif("test",dataDir,directory,0,3)

0
Flushing input history
Flushing output cache (0 entries)
1
Flushing input history
Flushing output cache (0 entries)
2
Flushing input history
Flushing output cache (0 entries)


In [9]:
%who

baseComparisonPlot	 baseGif	 dataDir	 directory	 hp	 imageio	 np	 os	 pd	 
plt	 readFile	 spatialPlot	 special	 temporalPlot	 


In [10]:
plt.clear()

AttributeError: module 'matplotlib.pyplot' has no attribute 'clear'

In [11]:
plt.clf()

<Figure size 432x288 with 0 Axes>

In [12]:
plt.show()

In [13]:
plt.close()